In [1]:
#Use 3.12.4
import openai
import os
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.docstore.document import Document
import tiktoken


key_path = '/Users/jaesolshin/key/openai_key.txt'
os.environ["OPENAI_API_KEY"] = open(key_path, 'r', encoding='utf-8').read()
openai.api_key = os.environ["OPENAI_API_KEY"] 

# Document Loaders

In [2]:
help(TextLoader.aload)

Help on function aload in module langchain_core.document_loaders.base:

async aload(self) -> 'List[Document]'
    Load data into Document objects.



In [3]:
path = '/Users/jaesolshin/Documents/GitHub/bokbot/rectified_text_2023.txt'
loader = TextLoader(path)
documents = loader.load()

In [4]:
print(f'문서개수: {len(documents)}')
print(f'첫번째 문서의 내용: {documents[0].page_content}')
print(f'첫번째 문서의 메타데이터: {documents[0].metadata}')

문서개수: 1
첫번째 문서의 내용: ISSN 1975-4922 2023 연차보고서 2024. 3｢한국은행법｣ 제102조(연차보고서의 공표) ① 한국은행은 매 회계연도가 지난 후 3개월 이내에 해당 회계연도 중의 한국은행의 업무상태와 통화 및 정부의 외환에 관한 정책을 개략적으로 기술하고 금융경제 상태를 분석한 연차보고서를 정부에 제출하고, 이를 공표하여야 한다. ② 제1항에 따른 연차보고서는 금융통화위원회의 의결을 거쳐야 한다.총재 서문 한국은행은 우리나라 중앙은행으로서 통화신용정책의 효율적 수행을 통해 물가안정과 금융안정을 도모함으로써 국민경제의 건전한 발전에 이바지하고자 최선을 다하고 있습니다. 2023년은 급변하는 안팎의 여건에 대응하여 보다 나은 정책과 해법을 마련하는데 분주했던 한 해였습니다. 인플레이션을 잡기 위한 주요국 중앙은행들의 가파른 금리인상에 이어 미국 실리콘밸리 은행사태, 이스라엘-하마스 전쟁 등으로 경계의 끈을 한시도 늦출 수 없는 긴장된 순간의 연속이었습니다. 이처럼 높아진 대외여건의 불확실성 속에서 한국은행은 최우선으로 물가안정을 도모하는 가운데 금융안정 측면의 리스크를 면밀히 점검하며 통화정책을 운영하였습니다. 기준금리를 3.5%의 긴축적인 수준으로 유지하고, 새마을금고 예금인출사태 등 금융･외환시장 불안에는 시장안정화조치를 통해 적극 대처하였습니다. 한국은행은 경제상황에 대한 정확한 진단과 정교한 정책대응을 뒷받침하고자 조사 및 정책연구업무를 적극적으로 수행하였습니다. 초저출산 및 초고령사회의 극단적인 인구구조문제, 장기구조적 관점에서 진단한 가계부채현황, 지역간 인구이동과 지역경제 등에 대한 보고서를 통해 우리나라가 해결해야 할 중장기 과제를 제시했습니다. 또한 기후변화 및 AI 확산 등 최신현안에 대해서도 다각적으로 분석하는 한편 지급결제 환경 변화에도 선제적으로 대응하기 위해 중앙은행 디지털화폐(CBDC) 활용성 테스트를 관계기관과 함께 추진하고 관련 연구를 이어나갔습니다. 한국은행은 국제사회에서 우리나라의 높아진 위상에 부

# Text Splitting

In [5]:
from langchain_text_splitters import CharacterTextSplitter

# tiktoken 기반 텍스트 스플리터 초기화
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=300, chunk_overlap=50)

# 문서를 분할
splitted_documents = text_splitter.split_documents(documents)

# 분할 결과 출력
print(f'분할 전 문서 개수: {len(documents)}')
print(f'분할 후 문서 개수: {len(splitted_documents)}')


Created a chunk of size 14675, which is longer than the specified 300
Created a chunk of size 335, which is longer than the specified 300
Created a chunk of size 577, which is longer than the specified 300
Created a chunk of size 3088, which is longer than the specified 300
Created a chunk of size 3526, which is longer than the specified 300
Created a chunk of size 3210, which is longer than the specified 300
Created a chunk of size 3610, which is longer than the specified 300
Created a chunk of size 426, which is longer than the specified 300
Created a chunk of size 521, which is longer than the specified 300
Created a chunk of size 983, which is longer than the specified 300
Created a chunk of size 305, which is longer than the specified 300
Created a chunk of size 619, which is longer than the specified 300
Created a chunk of size 303, which is longer than the specified 300
Created a chunk of size 310, which is longer than the specified 300
Created a chunk of size 5361, which is lon

분할 전 문서 개수: 1
분할 후 문서 개수: 158


# Vector DB

In [6]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import time

persist_directory = 'chroma/'

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')

# Chroma 데이터베이스 초기화
database = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings
)

# 배치 크기 설정
batch_size = 100
splitted_documents_batches = [splitted_documents[i:i + batch_size] for i in range(0, len(splitted_documents), batch_size)]

for i, batch in enumerate(splitted_documents_batches[:10]):
    print(f"Batch {i+1}/{len(splitted_documents_batches)} 처리 중...")
    database.add_documents(documents=batch)  # 현재 배치를 기존 DB에 추가
    print(f"Batch {i+1} 처리 완료.")
    # API 제한을 피하기 위해 딜레이 추가
    time.sleep(1)

database = Chroma.from_documents( #Chroma를 초기화
    documents = splitted_documents, #추가할 문서 지정
    persist_directory = persist_directory, #저장경로 지정
    embedding = embeddings #벡터화할 모델을 지정
)

print('DB 생성이 완료되었습니다.')

/var/folders/hw/9m3g7fvn4_l3rp2y473km9sm0000gn/T/ipykernel_5603/1448614959.py:8: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')
/var/folders/hw/9m3g7fvn4_l3rp2y473km9sm0000gn/T/ipykernel_5603/1448614959.py:11: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  database = Chroma(


Batch 1/2 처리 중...
Batch 1 처리 완료.
Batch 2/2 처리 중...
Batch 2 처리 완료.


In [ ]:
#분할된 문장을 벡터db에 저장
#!pip install tiktoken
#!pip install chromadb==0.5.3 
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

persist_directory = 'chroma/'

embeddings = OpenAIEmbeddings( #임베딩 모델을 초기화
    model = 'text-embedding-ada-002' #모델명 지정
)

database = Chroma.from_documents( #Chroma를 초기화
    documents = splitted_documents, #추가할 문서 지정
    persist_directory = persist_directory, #저장경로 지정
    embedding = embeddings #벡터화할 모델을 지정
)

print('DB 생성이 완료되었습니다.')

DB 생성이 완료되었습니다.


# Retrieval

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = OpenAIEmbeddings(
    model = 'text-embedding-ada-002'
)

database = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings
)

documents = database.similarity_search('올해 한국은행의 정책목표 중 가장 중요한 것은 무엇인가? 답변과 함께 근거를 제시하라')

print(f'문서 개수: {len(documents)}')

for d in documents:
    print(f'문서 내용: {d.page_content}')

문서 개수: 4
문서 내용: 부록 (4) 금융안정회의 주요 내용 ◇ ｢금융안정 상황점검｣ 주요 논의 내용 (2023년 3월 23일, 제6차) (1) 금융안정국장이 보고 제20호–｢금융안정 상황점검｣에 대하여 보고하였음. (2) 위원 토의 내용 관련 부서는 3월 15일 위원협의회에서 논의된 주요 내용을 다음과 같이 보고하였음. 우선 여러 위원들은 금번 보고서가 새로운 분석방법을 통해 금융안정 관련 유용한 정보들을 제공하는 등 분석의 정도가 향상되었다고 평가하였음. 또한 실리콘밸리은행 파산 사태 등 최근 이슈도 속보성 있게 다뤄 보고서의 시의성을 높인 점도 긍정적이라는 의견도 제시하였음. 다만 보고서가 핵심내용 위주로 간결하게 작성되어 전달력이 높아진 측면도 있으나 일부 파트의 경우 내용이 지나치게 축약되어 독자의 이해도 제고 차원에서 추가 설명이 필요하다는 견해를 나타내었음. 일부 위원은 최근의 실리콘밸리은행 파산 사태 분석 사례를 참조하여 향후 보고서의 속보성을 높이기 위해 기존 보고서의 분석 시계에 대해 고민할 필요가 있다는 의견을 주었음. 또한 일부 위원은 보고서 작성에 시간 제약은 있으나 실리콘밸리은행 파산 사태가 금융안정 상황에 미칠 파급력을 감안할 때 관련 내용을 보다 심도 있게 볼 필요가 있다고 언급하였음. 아울러 일부 위원은 현재 우리나라 민간 신용의 과다 여부를 판단하는 기준의 하나로 과거 추세와 비교하고 있는데, 그간의 경제성장 속도 및 부채 증가세 등이 지속될지 여부가 불확실한 상황임을 감안할 때 여타 다양한 지표도 살펴볼 필요가 있다는 견해를 밝혔음. 한편 일부 위원은 최근 국제기구인 BIS 기준의 가계부채 DSR 지표가 언론에서 많이 다뤄지고 있는데, 우리나라 기준의 DSR 지표와 자칫 혼동을 야기할 수 있으므로 산정 방식 등 차이점에 대해 보다 자세하게 설명할 필요가 있다는 의견을 나타내었음. 이 밖에 일부 위원은 동 보고서가 유용한 정보를 충실히 제공하고 있어 자료 업데이트 및 추가 분석 가능 여부를 검토하여 통화정책 방향 참고자료 및

# Query Augmentation

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

chat = ChatOpenAI(model='gpt-3.5-turbo')

embeddings = OpenAIEmbeddings(
    model = 'text-embedding-ada-002'
)

database = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings
)

retriever = database.as_retriever() #데이터베이스를 Retriever로 변환

qa = RetrievalQA.from_llm( #RetrievalQA를 초기화
    llm=chat, #Chat models를 지정
    retriever = retriever, #Retriever를 지정
    return_source_documents=True #응답에 원본 문서를 포함할지 결정
)
#RetrievalQA를 사용하면 출처가 되는 문서들을 결합하는 방법도 선택할 수 있다
#단순결합, Refine, Map Reduce, Map re-rank

result = qa('올해 한국은행의 정책목표 중 가장 중요한 것은 무엇인가?')

print(result['result']) #응답을 표시
print(result['source_documents']) #원본 문서를 표시

InvalidRequestError: This model's maximum context length is 16385 tokens. However, your messages resulted in 50938 tokens. Please reduce the length of the messages.